In [2]:
from scPreGAN.model.util import load_anndata
from thesis import DATA_PATH, SAVED_RESULTS_PATH
from scPreGAN import Model

In [3]:
data_path = DATA_PATH / "pbmc" / "pbmc.h5ad"

In [4]:
condition_key = 'condition'
condition = {
    'case': 'stimulated',
    'control': 'control'
}
cell_type_key = 'cell_type'
out_of_sample_prediction = True
cell_type= None


adata_split, train_data = load_anndata(path=data_path,
                condition_key=condition_key,
                condition=condition,
                cell_type_key=cell_type_key,
                out_sample_prediction=out_of_sample_prediction,
                prediction_type=cell_type
                )
control_adata, perturb_adata = adata_split
control_pd, control_celltype_ohe_pd, perturb_pd, perturb_celltype_ohe_pd = train_data

In [5]:
# create model
n_features = control_pd.shape[1]
n_classes = control_adata.obs[cell_type_key].unique().shape[0]
print(n_features, n_classes)

model = Model(n_features=n_features, n_classes=n_classes, use_cuda=True)


6998 7
Random Seed:  3060
Successfully created the model


In [10]:
# training
model_path = SAVED_RESULTS_PATH / "scPreGAN"
model.train(train_data=train_data)

[100/20000] adv_loss: 0.0063  recon_loss: 0.0566 encoding_loss: 0.0023 G_loss: 0.0652 D_A_loss: -0.1936  D_B_loss: 0.0014
[200/20000] adv_loss: 0.0064  recon_loss: 0.0552 encoding_loss: 0.0009 G_loss: 0.0625 D_A_loss: -0.5743  D_B_loss: 0.2479
[300/20000] adv_loss: 0.0060  recon_loss: 0.0521 encoding_loss: 0.0008 G_loss: 0.0589 D_A_loss: -0.6406  D_B_loss: -0.0280
[400/20000] adv_loss: 0.0099  recon_loss: 0.0484 encoding_loss: 0.0007 G_loss: 0.0590 D_A_loss: -0.9805  D_B_loss: -0.3194
[500/20000] adv_loss: 0.0113  recon_loss: 0.0503 encoding_loss: 0.0007 G_loss: 0.0623 D_A_loss: -1.1277  D_B_loss: -0.4054
[600/20000] adv_loss: 0.0108  recon_loss: 0.0475 encoding_loss: 0.0006 G_loss: 0.0589 D_A_loss: -1.1619  D_B_loss: -0.5453
[700/20000] adv_loss: 0.0132  recon_loss: 0.0482 encoding_loss: 0.0007 G_loss: 0.0621 D_A_loss: -1.1946  D_B_loss: -0.6219
[800/20000] adv_loss: 0.0131  recon_loss: 0.0462 encoding_loss: 0.0008 G_loss: 0.0601 D_A_loss: -1.3608  D_B_loss: -0.6784
[900/20000] adv_lo

In [ ]:
# predicting
pred_perturbed_adata = model.predict(control_adata=control_adata,
                   cell_type_key=cell_type_key,
                   condition_key=condition_key)